Welcome to Snowflake! This guide shows how to fine-tune a foundational LLM (Large Language Model) using Cortex Serverless SQL functions. 

In this exercise, you will:

* Use `mistral-large` model to categorize customer support tickets
* Prepare training data for fine-tuning using `mistral-7b` to generate annotations
* Fine-tune `mistral-7b` to achieve the accuracy of `mistral-large` at fraction of cost
* Generate custom email copy for each support ticket using the fine-tuned model

## Import Snowpark and create Snowpark session

In [ ]:
import snowflake.snowpark.functions as F
import streamlit as st
import altair as alt

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()
# Add a query tag to the session. This helps with troubleshooting and performance monitoring.
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"aiml_notebooks_fine_tuning", 
                     "version":{"major":1, "minor":0},
                     "attributes":{"is_quickstart":1, "source":"notebook"}}

## Load customer support ticket data from AWS S3 into a Snowflake table
This section walks you through the steps to:

- Create a database and schema.
- Create a file format for the data.
- Create an external stage.
- Create a table.
- Load the data from external stage.

In [ ]:
CREATE OR REPLACE DATABASE VINO_DB;
CREATE OR REPLACE SCHEMA VINO_SCHEMA;
USE SCHEMA VINO_DB.VINO_SCHEMA;

In [ ]:
CREATE or REPLACE file format csvformat
  SKIP_HEADER = 1
  FIELD_OPTIONALLY_ENCLOSED_BY = '"'
  type = 'CSV';

CREATE or REPLACE stage support_tickets_data_stage
  file_format = csvformat
  url = 's3://sfquickstarts/finetuning_llm_using_snowflake_cortex_ai/';

In [ ]:
CREATE or REPLACE TABLE SUPPORT_TICKETS (
  ticket_id VARCHAR(60),
  customer_name VARCHAR(60),
  customer_email VARCHAR(60),
  service_type VARCHAR(60),
  request VARCHAR,
  contact_preference VARCHAR(60)
);

In [ ]:
COPY into SUPPORT_TICKETS
  from @support_tickets_data_stage;

In [ ]:
df_support_tickets = session.table('support_tickets')
df_support_tickets.show()

## Categorize Support Tickets: 
By prompting both `mistral-large` and `mistral-7b` models, let's categorize the customer support tickets into one of 5 classes, based on the complaints.

- Roaming fees
- Slow data speed
- Lost phone
- Add new line
- Closing account

In [ ]:
prompt = """You are an agent that helps organize requests that come to our support team. 

The request category is the reason why the customer reached out. These are the possible types of request categories:

Roaming fees
Slow data speed
Lost phone
Add new line
Closing account

Try doing it for this request and return only the request category only.
"""

## Let's use `mistral-large` to categorize the tickets.

In [ ]:
mistral_large_response_sql = f""" select ticket_id, 
                                        request, 
                                        trim(snowflake.cortex.complete('mistral-large',
                                                                        concat('{prompt}',
                                                                        request)),'\n') as mistral_large_response
                                    from support_tickets
                                """

df_mistral_large_response = session.sql(mistral_large_response_sql)
df_mistral_large_response.show()

## Let's now use `mistral-7b` to categorize the tickets.

In [ ]:
mistral_7b_response_sql = f""" select ticket_id,
                                    trim(snowflake.cortex.complete('mistral-7b',
                                                                        concat('{prompt}',
                                                                        request)),'\n') as mistral_7b_response
                                from support_tickets
                            """

df_mistral_7b_response = session.sql(mistral_7b_response_sql)
df_mistral_7b_response.show()

## Let's compare the categorization results of both models

As you can see in the results below, the `mistral-large` does a good job of returning the ticket categories only. However, the `mistral-7b` returns additional text which is not the expected behavior.

Can we fine-tune `mistral-7b` to achieve better accuracy instead of using a larger model?

In [ ]:
df_llms = df_mistral_large_response.join(df_mistral_7b_response,'ticket_id')
df_llms.show()

## Prepare/ Generate dataset to fine-tune `mistral-7b`

- For the next step, let's use `mistral-large` model to categorize the support tickets, and create training dataset from the model responses. 

- Let us then use this dataset to fine-tune the smaller `mistral-7b` model.

- The annotated dataset is saved into `support_tickets_finetune` table in Snowflake.

In [ ]:
df_fine_tune = df_mistral_large_response.with_column("prompt", 
                                                     F.concat(F.lit(prompt),F.lit(" "),F.col("request"))).\
                                        select("ticket_id","prompt","mistral_large_response")

df_fine_tune.write.mode('overwrite').save_as_table('support_tickets_finetune')

In [ ]:
train_df, eval_df = session.table("support_tickets_finetune").random_split(weights=[0.8, 0.2], seed=42)
train_df.write.mode('overwrite').save_as_table('support_tickets_train')
eval_df.write.mode('overwrite').save_as_table('support_tickets_eval')

In [ ]:
session.table('support_tickets_train').show(1)

In [ ]:
session.table('support_tickets_eval').show(1)

## Fine-tune `mistral-7b` using Cortex

Let's fine-tune using the annotated dataset from `support_tickets_finetune` table

- Use `snowflake.cortex.finetune()` to run the fine-tuning job
- Monitor progress
- Run inference on the fine-tuned model

In [ ]:
select snowflake.cortex.finetune('CREATE', 
                                    'VINO_DB.VINO_SCHEMA.SUPPORT_TICKETS_FINETUNED_MISTRAL_7B', 
                                    'mistral-7b', 
                                    'SELECT prompt, mistral_large_response as completion from VINO_DB.VINO_SCHEMA.support_tickets_train', 
                                    'SELECT prompt, mistral_large_response as completion from VINO_DB.VINO_SCHEMA.support_tickets_eval');

To see the progress of the fine-tuning job, copy the `job id` from the above cell result and update the second parameter of the `finetune()` function.

In [ ]:
select snowflake.cortex.finetune('DESCRIBE', 'CortexFineTuningWorkflow_3b54b820-7173-4a07-83ad-5645bd4c45ec');

## Inference using fine-tuned model 

Let's use this fine-tuned `mistral-7b` model that we named `SUPPORT_TICKETS_FINETUNED_MISTRAL_7B` on the eval dataset to categorize the tickets.

In [ ]:
fine_tuned_model_name = 'SUPPORT_TICKETS_FINETUNED_MISTRAL_7B'
fine_tuned_response_sql = f"""
        select ticket_id, 
            request,
            trim(snowflake.cortex.complete('{fine_tuned_model_name}',concat('{prompt}',request)),'\n') as fine_tuned_mistral_7b_model_response
        from support_tickets
        """

df_fine_tuned_mistral_7b_response = session.sql(fine_tuned_response_sql)
df_fine_tuned_mistral_7b_response

Let's visualize the ticket categories and the number of tickets per category

In [ ]:
df = df_fine_tuned_mistral_7b_response.group_by('fine_tuned_mistral_7b_model_response').\
                                        agg(F.count("*").as_('COUNT'))

st.subheader("Number of requests per category")
chart = alt.Chart(df.to_pandas()).mark_bar().encode(
    y=alt.Y('FINE_TUNED_MISTRAL_7B_MODEL_RESPONSE:N', sort="-x"),
    x=alt.X('COUNT:Q',),
    color=alt.Color('FINE_TUNED_MISTRAL_7B_MODEL_RESPONSE:N', scale=alt.Scale(scheme='category10'), legend=None),
).properties(height=400)

st.altair_chart(chart, use_container_width=True)

## Streamlit application to auto-generate custom emails and text messages

Since we are able to rightly categorize the customer support tickets based on root cause, the next step is to auto-generate custom email responses for each support ticket.

Let's build a Streamlit app that allows us to choose between these 4 LLMs to generate the email copy:
- `snowflake-arctic`
- `llama3-8b`
- `mistral-large`
- `reka-flash`

In [ ]:
st.subheader("Auto-generate custom emails or text messages")

with st.container():
    with st.expander("Edit prompt and select LLM", expanded=True):
        entered_prompt = st.text_area('Prompt',"""Please write an email or text promoting a new plan that will save customers total costs. If the customer requested to be contacted by text message, write text message response in less than 25 words, otherwise write email response in maximum 100 words.""")
    
        with st.container():
            left_col,right_col = st.columns(2)
            with left_col:
                selected_category = st.selectbox('Select category',('Roaming fees', 'Closing account', 'Add new line', 'Slow data speed'))
            with right_col:
                selected_llm = st.selectbox('Select LLM',('snowflake-arctic','llama3-8b','mistral-large', 'reka-flash',))

with st.container():
    _,mid_col,_ = st.columns([.4,.3,.3])
    with mid_col:
        generate_template = st.button('Generate messages ⚡',type="primary")

with st.container():
    if generate_template:
        sql = f"""select s.ticket_id, s.customer_name, concat(IFF(s.contact_preference = 'Email', '📩', '📲'), ' ', s.contact_preference) as contact_preference, snowflake.cortex.complete('{selected_llm}',
        concat('{entered_prompt}','Here is the customer information: Name: ',customer_name,', Contact preference: ', contact_preference))
        as llm_response from support_tickets as s join support_tickets_train as t on s.ticket_id = t.ticket_id
        where t.mistral_large_response = '{selected_category}' limit 10"""

        with st.status("In progress...") as status:
            df_llm_response = session.sql(sql).to_pandas()
            st.subheader("LLM-generated emails and text messages")
            for row in df_llm_response.itertuples():
                status.caption(f"Ticket ID: `{row.TICKET_ID}`")
                status.caption(f"To: {row.CUSTOMER_NAME}")
                status.caption(f"Contact through: {row.CONTACT_PREFERENCE}")
                status.markdown(row.LLM_RESPONSE.replace("--", ""))
                status.divider()
            status.update(label="Done!", state="complete", expanded=True)

You have learnt how to finetune an Large Language Model using Snowflake Cortex. To learn more about Cortex and LLMs, please check out: https://developers.snowflake.com/solutions/?_sft_technology=snowflake-cortex
